In [1]:
import numpy as np
from gensim.models import Word2Vec
import json
import copy
from collections import Counter

In [ ]:
###copied over from HW2 assignment, lazy way of getting a corpus with UNKS
class Dataset:
    def __init__(self, link):
        self.all_tweets=self.read_data(link)
        #if train data, get all the necessary things
        self.clean_tweets, self.vocab, self.pairs=self.get_clean_datasets()
        self.only_words=[[pair[0] for pair in tweet] for tweet in self.clean_tweets]
            
    def read_data(self,link):
        #read in the stuff
        all_tweets=[]
        tot_time=0
        for line in open(link):
            curr_dat=json.loads(line)
            all_tweets.append(curr_dat)
        return all_tweets
    
    def get_clean_datasets(self):
        vocab=Counter([c[0] for d in self.all_tweets for c in d])
        #cleaned word/tag combos and cleaned tweets:
        vocab, clean_tweets=self.add_unks(vocab,3)
        pairs=Counter([(a[0],a[1]) for b in clean_tweets for a in b])
        return clean_tweets,vocab,pairs
        

#we need two things in this function:
#1. clean the word vocabulary
#2. replace words for UNKs in the word/tag pairs for calculaing e(x|y) later 
    def add_unks(self,vocab, thresh):
        all_words=vocab.copy()
        low_freq_words=[k for k,v in all_words.items() if v <= thresh]
        ###
        print('Fraction of total vocabulary replaced with UNKs', len(low_freq_words)/len(all_words))
        ddd=sum([all_words[l] for l in low_freq_words])/sum(all_words.values())
        print('Fraction of total corpus replaced with UNKs',ddd)
        unks=[]
        for word in low_freq_words:
            unks+=[self.check_unk_type(word)]*all_words[word]
            del all_words[word]
        all_words+=Counter(unks)
        #clean and count the word/tag pairs
        clean_tweets=self.cleanse_tweets(all_words)
        return all_words, clean_tweets
    
    def cleanse_tweets(self,vocab):
        clean_tweets=copy.deepcopy(self.all_tweets)
        for line in clean_tweets:
            for pair in line:
                if vocab[pair[0]]==0:
                    pair[0]=self.check_unk_type(pair[0])
        return clean_tweets
        

    def check_unk_type(self,word):
        if(word.isdigit()):
            if(len(word)==4 and (word.startswith('1') or word.startswith('2'))):
                return 'UNK_YEAR'
            else:
                return 'UNK_NUMBER'
        elif((len(word)>=3 and word[-3]=='.' and word[-2:].isdigit()) or word[0]=='$' or word[-1]=='$'):
            return 'UNK_MONEY'
        elif(len(word)>=8 and word[:2].isdigit() and word[2]=='/' and word[3:5].isdigit() and word[5]=='/' and word[6:].isdigit()):
            return 'UNK_DATE'
        elif(word[0]=='#'):
            return 'UNK_HASH'
        elif(word[0]=='@'):
            return 'UNK_AT'
        elif(word[:4]=='http' or word[:4]=='www.'):
            return 'UNK_LINK'
        elif(len(word)==1 and f'U+{ord(word):X}'[:4]=='U+1F'):
            return 'UNK_EMOJI'
        elif(word.isupper()):
            return 'UNK_CAPS'
        elif(word[0].isupper()):
            return 'UNK_UPCASE'
        elif(word.islower()):
            return 'UNK_LOWCASE'
        else:
            return 'UNK_ELSE'

In [ ]:
data=Dataset('../HW2/hw2-data/twt.bonus.json')

In [ ]:
clean_corp=data.only_words
model=Word2Vec(clean_corp,workers=8,sg=1)

In [ ]:
model.save('twitter_opt_try_one.bin')

In [ ]:
#visualize?
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

X=model[model.wv.vocab]
pca=PCA(n_components=2)
result=pca.fit_transform(X)
words = list(model.wv.vocab)

plt.figure(figsize=(15,15))
plt.scatter(result[:100,0],result[:100,1])
for i, word in enumerate(words[:100]):
    plt.annotate(word, xy=(result[i, 0], result[i, 1]))

In [ ]:
pca.explained_variance_